In [ ]:
import sys
sys.path.append('/home/tony/workspace/python/grad/resources/Biocyc-Facade/src')
sys.path = list(set(sys.path))
import os
import sqlite3

In [ ]:
con = sqlite3.connect('../test/scratch.pgdb')
cur = con.cursor()

In [ ]:
cur.execute(f''' \
CREATE TABLE json_keys_of_dats (
    key TEXT NOT NULL,
    table_name TEXT NOT NULL,
    PRIMARY KEY (key),
    FOREIGN KEY table_name REFERENCES tables (name)
)''')

In [ ]:
# sqlite gives this automatically
# CREATE TABLE sqlite_schema (
#   type TEXT,
#   name TEXT,
#   tbl_name TEXT,
#   rootpage INTEGER,
#   sql TEXT
# );
for x in cur.execute('select names, type, sql from sqlite_schema where name="test"'):
    print(x)

In [ ]:
con.close()

In [ ]:
import sys
sys.path.append('/home/tony/workspace/python/grad/resources/Biocyc-Facade/src')
sys.path = list(set(sys.path))

import os
from biocyc_facade.pgdb import ImportFromBiocyc, Pgdb, Dat
from biocyc_facade.models.sql import TraceStep

DB_DIR = '/home/tony/workspace/python/grad/resources/Biocyc-Facade/test'
ECO = f'{DB_DIR}/eco'

os.system(f'rm {ECO}.pgdb')
edb = ImportFromBiocyc(ECO, '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5/ecoli/25.1/data')

# edb = Pgdb(ECO)
# edb.GetInfo()

In [ ]:
dbn = 'ecocyc'
db_path = f'../test/{dbn}'
flat_files = '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5/ecoli/25.1/data'
# os.system(f'rm ../test/{dbn}.pgdb')

# db = ImportFromBiocyc(db_path, flat_files)
db = Pgdb(db_path)

In [ ]:
tbs = db.registry.GetTable('proteins_dat')
assert tbs is not None

In [ ]:
ents = db.ListEntries(Dat.GENES)
ents[124]

In [ ]:
db.GetEntry(Dat.PROTEINS, ents[123])

In [ ]:
cur = db._database._cur

js = []
for i in range(20):
    for p in x:
        j = cur.execute(f"select json from proteins_dat where id='{p}'")
        js.append(j)

len(js)

In [ ]:
jss = []
for i in range(20):
    js = list(cur.execute(f"select json from proteins_dat"))
    jss += js

len(jss)

In [ ]:
def _makeMappingPlan(source: Dat, dest: Dat):
        def usekey(key):
            return lambda v: v.get(key, list())
        passthrough = lambda v: v

        # from: (to, table, method)
        RELATIONS = {
            Dat.PATHWAYS: [
                (Dat.REACTIONS, Dat.PATHWAYS, usekey('REACTION-LIST')),
            ],
            Dat.REACTIONS: [
                (Dat.PATHWAYS, Mapping.RXN_PATHWAY, passthrough),
                (Dat.ENZRXNS, Dat.REACTIONS, usekey('ENZYMATIC-REACTION')),
            ],
            Dat.ENZRXNS: [
                (Dat.REACTIONS, Dat.ENZRXNS, usekey('REACTION')),
                (Dat.PROTEINS, Dat.ENZRXNS, usekey('ENZYME')),
            ],
            Dat.PROTEINS: [
                (Dat.ENZRXNS, Mapping.PROT_ENZRXN, passthrough),
            ]
        }

        def find(curr:Dat, path):
            if curr == dest:
                print('x')
                return path
            for to, table, fn in RELATIONS.get(curr, []):
                return find(to, path+[(table, fn)])
            return None

        path = find(source, [])
        assert path is not None, f"no known way to get from {source.name} to {dest.name}"
        for t, fn in path:
            print(t, fn)

_makeMappingPlan(Dat.PROTEINS, Dat.PATHWAYS)

In [ ]:
q = ['1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER',
 '1-PFK',
 '1-PFK-MONOMER',
 '2-DEHYDROPANTOATE-REDUCT-MONOMER',
 '2-HEXADECENOYL-ACP',
 '2-ISOPROPYLMALATESYN-MONOMER',
 '2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER',
 '2-OCTAPRENYLPHENOL-HYDROX-MONOMER',
 '2OXOGLUTARATEDEH-CPLX',
 '3-CH3-2-OXOBUTANOATE-OH-CH3-XFER-MONOMER']


cur.execute(f'''create temporary table temp (x text primary key)''')
cur.execute(f'''create temporary table temp (x text primary key)''')

In [ ]:
db.GetInfo()

In [ ]:
from __future__ import annotations
from enum import Enum

class Dat(Enum):
    PROTEINS = 'proteins.dat'
    ENZRXNS = 'enzrxns.dat'
    PATHWAYS = 'pathways.dat'
    GENES = 'genes.dat'
    REGULATION = 'regulation.dat'
    REACTIONS = 'reactions.dat'
    COMPOUNDS = 'compounds.dat'
    PROTEIN_FEATURES = 'protein-features.dat'
    PRO_LIGAND_CPLX = 'protligandcplxes.dat'
    RNAS = 'rnas.dat'
    SPECIES = 'species.dat'

    def __init__(self, val: str):
        self.table_name = val.replace('.', '_').replace('-', '_')

In [ ]:
Dat.COMPOUNDS.table_name

In [ ]:
verp = f'{flat_files}/version.dat'
import json
from typing import Any

def jdumps(obj: Any):
    return json.dumps(obj, separators=(',', ':'))

jloads = json.loads
# ORGID	META
# ORGANISM	MetaCyc
# VERSION	25.5
# RELEASE-DATE	Wed Jul 21, 2021

def parse(fpath: str):
    VER_DAT = 'version.dat'
    version_data = {}
    with open(fpath) as ver:
        for line in ver:
            if line.startswith(';;') or line.startswith('//'): continue
            toks = line[:-1].split('\t')
            if len(toks) != 2:
                print(f'unrecognized line in {VER_DAT} [{line}]')
                continue
            k, v = toks
            vals = version_data.get(k, list())
            vals.append(v)
            version_data[k] = vals
        version_data = [(k, v[0] if len(v)==1 else jdumps(v)) for k, v in version_data.items()]
    
    assert len(version_data)>0, f'{VER_DAT} is empty!'
    return version_data

vd = parse(verp)

In [ ]:
vd

In [ ]:
[t.name for t in db._database.registry.GetTables()]

In [ ]:
for t in db._database.registry.GetTables():
    print(t.name)

In [ ]:
db.GetInfo()

In [ ]:
r = db._database.registry
def x():
    for name, fields, t in r.Select([r.F_table_name, r.F_fields, r.F_type], ''):
        yield name
for xx in x():
    print(xx)